In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from joblib import dump

from neuropixel_analysis.lib.plotting_utils import raster

plt.rc('font', size=14)
%matplotlib inline

<img src='2AFC_events.png' width=1200 height=1200/>

In [ ]:
behav_df = pd.DataFrame()
behav_df['Rewarded'] = [1, 0, 0]
behav_df['CorrectChoice'] = [1, 0, 0]
behav_df['CompletedTrial'] = [1, 1, 0]
behav_df["ChosenDirection"] = [1, 2, -1]
behav_df['TrialNumber'] = [0, 1, 2]
behav_df['WaitingTime'] = [3.1401, 1.0460, 2.2079]
behav_df.loc[:, 'TrialStartAligned'] = [10.187, 22.253, 35.293]  # seconds from beginning of recording (only really care about difference)

behav_df['DV'] = [-0.15556, 0.948718, -0.2]
behav_df['stim_dir'] = [1, 0, 1]

behav_df['CatchTrial'] = [False, False, False]

behav_df['PokeCenterStart'] = [6.4965, 4.5105, 8.4341]
behav_df['StimulusOnset'] = [6.7186, 4.9032, 8.6681]
behav_df['StimulusOffset'] = [7.0687, 5.2533, 9.0182]

behav_df['ResponseStart'] = [7.3495, 5.4722, 9.3196]
behav_df['ResponseEnd'] = [10.9949, 8.6123, 10.3656]

In [ ]:
n_trials = len(behav_df)
print(n_trials)
behav_df.head()

In [ ]:
n_nrns = 100
recording_len = 50  # seconds
ms = 1000
spike_mat_ms = np.random.choice([0, 1], size=(n_nrns, recording_len * ms), p=[0.999, 0.001]).astype('uint8')

In [ ]:
raster(spike_mat_ms, begin_bin=8*1000, end_bin=recording_len*1000, sort=False)
for i in range(n_trials):
    _start = behav_df.TrialStartAligned[i] * ms
    plt.axvline(_start, c='C0')
    plt.axvline(_start + behav_df.StimulusOnset[i] * ms, c='C1')
    plt.axvline(_start + behav_df.ResponseStart[i] * ms, c='C2')
    plt.axvline(_start + behav_df.ResponseEnd[i] * ms, c='C3')

    t_arr = np.arange(_start, _start + behav_df.ResponseEnd[i] * ms)
    plt.fill_between(x=t_arr, y1=np.ones_like(t_arr), y2=100 * np.ones_like(t_arr), color='C4', alpha=0.2)


In [ ]:
t_event = np.arange(-1000., 1000.)
len_arch = t_event.shape[0]
stim_arch = np.exp(-t_event**2 / (2 * 1e5)) * 0.01

zero_point = int(len_arch/2)
resp_ramp = np.zeros_like(t_event)
resp_ramp[:zero_point] = (t_event[:zero_point] + 1000) * 1e-5
resp_ramp[zero_point:int(1.25*zero_point)] = 0.01 - t_event[zero_point:int(1.25*zero_point)]/25000

# resp_ramp *= 10
# stim_arch *= 10

In [ ]:
plt.subplot(121)
plt.plot(t_event, stim_arch)
plt.subplot(122)
plt.plot(t_event, resp_ramp)


In [ ]:
stim_clust_size = int(n_nrns / 4)

In [ ]:
spiking_around_stim = np.zeros((stim_clust_size, len_arch))
ramp_to_stim = np.zeros((stim_clust_size, len_arch))
ramp_to_resp = np.zeros((stim_clust_size, len_arch))
for i in range(len_arch):
    spiking_around_stim[:, i] = np.random.choice(a=[0, 1], size=stim_clust_size, p=[1 - stim_arch[i], stim_arch[i]])
    # ramp_to_stim[:, i] = np.random.choice(a=[0, 1], size=stim_clust_size, p=[1 - 0.5 * resp_ramp[i], 0.5*resp_ramp[i]])
    ramp_to_resp[:, i] = np.random.choice(a=[0, 1], size=stim_clust_size, p=[1 - resp_ramp[i], resp_ramp[i]])


In [ ]:
plt.subplot(131)
raster(spiking_around_stim, 0, 2000, sort=False)
plt.subplot(132)
raster(ramp_to_stim, 0, 2000, sort=False)
plt.subplot(133)
raster(ramp_to_resp, 0, 2000, sort=False)


In [ ]:
stim_arch_mat = (np.repeat(stim_arch[:, np.newaxis], stim_clust_size, axis=1).transpose(1, 0) * 1000).astype('uint8')
plt.plot(stim_arch_mat[0])
stim_arch_mat.shape

In [ ]:
resp_ramp_mat = (np.repeat(resp_ramp[:, np.newaxis], stim_clust_size, axis=1).transpose(1, 0)*1000).astype('uint8')
plt.plot(resp_ramp_mat[1])

In [ ]:
SMOOTH_TRACES = False
if SMOOTH_TRACES:
    updated_spike_mat = np.zeros_like(spike_mat_ms)
    for i in range(n_trials):
        _start = behav_df.TrialStartAligned[i] * ms
        stim_on = int(_start + behav_df.StimulusOnset[i] * ms)
        resp_end = int(_start + behav_df.ResponseEnd[i] * ms)
        if i%2:
            updated_spike_mat[int(3*stim_clust_size):, (stim_on - 1000):(stim_on+1000)] = stim_arch_mat
            updated_spike_mat[:int(stim_clust_size), (resp_end - 1000):(resp_end+1000)] = resp_ramp_mat
        else:
            updated_spike_mat[int(2*stim_clust_size):int(3*stim_clust_size), (stim_on - 1000):(stim_on+1000)] = stim_arch_mat
            updated_spike_mat[int(stim_clust_size):int(2*stim_clust_size), (resp_end - 1000):(resp_end+1000)] = resp_ramp_mat
else:
    updated_spike_mat = spike_mat_ms.copy()
    for i in range(n_trials):
        _start = behav_df.TrialStartAligned[i] * ms
        stim_on = int(_start + behav_df.StimulusOnset[i] * ms)
        resp_end = int(_start + behav_df.ResponseEnd[i] * ms)
        if i%2:
            updated_spike_mat[int(3*stim_clust_size):, (stim_on - 1000):(stim_on+1000)] = spiking_around_stim
            updated_spike_mat[:int(stim_clust_size), (resp_end - 1000):(resp_end+1000)] = ramp_to_resp
        else:
            updated_spike_mat[int(2*stim_clust_size):int(3*stim_clust_size), (stim_on - 1000):(stim_on+1000)] = spiking_around_stim
            updated_spike_mat[int(stim_clust_size):int(2*stim_clust_size), (resp_end - 1000):(resp_end+1000)] = ramp_to_resp

updated_spike_mat.shape, updated_spike_mat.sum()

In [ ]:
plt.figure(figsize=(18,8))
plt.subplot(131)
raster(updated_spike_mat, begin_bin=8*1000, end_bin=recording_len*1000, sort=False)
for i in range(n_trials):
    _start = behav_df.TrialStartAligned[i] * ms
    plt.axvline(_start, c='C0')
    plt.axvline(_start + behav_df.StimulusOnset[i] * ms, c='C1')
    plt.axvline(_start + behav_df.ResponseStart[i] * ms, c='C2')
    plt.axvline(_start + behav_df.ResponseEnd[i] * ms, c='C3')

    t_arr = np.arange(_start, _start + behav_df.ResponseEnd[i] * ms)
    plt.fill_between(x=t_arr, y1=np.ones_like(t_arr), y2=100 * np.ones_like(t_arr), color='C4', alpha=0.2)

plt.subplot(132)
raster(updated_spike_mat, begin_bin=8*1000, end_bin=recording_len*1000, sort=False)
_start = behav_df.TrialStartAligned[0] * ms
plt.axvline(_start, c='C0')
plt.axvline(_start + behav_df.StimulusOnset[0] * ms, c='C1')
plt.axvline(_start + behav_df.ResponseStart[0] * ms, c='C2')
plt.axvline(_start + behav_df.ResponseEnd[0] * ms, c='C3')

t_arr = np.arange(_start, _start + behav_df.ResponseEnd[0] * ms)
plt.fill_between(x=t_arr, y1=np.ones_like(t_arr), y2=100 * np.ones_like(t_arr), color='C4', alpha=0.2)
plt.xlim(10000, 22000)

plt.subplot(133)
raster(updated_spike_mat, begin_bin=8*1000, end_bin=recording_len*1000, sort=False)
_start = behav_df.TrialStartAligned[1] * ms
plt.axvline(_start, c='C0')
plt.axvline(_start + behav_df.StimulusOnset[1] * ms, c='C1')
plt.axvline(_start + behav_df.ResponseStart[1] * ms, c='C2')
plt.axvline(_start + behav_df.ResponseEnd[1] * ms, c='C3')

t_arr = np.arange(_start, _start + behav_df.ResponseEnd[1] * ms)
plt.fill_between(x=t_arr, y1=np.ones_like(t_arr), y2=100 * np.ones_like(t_arr), color='C4', alpha=0.2)
plt.xlim(23000, 32000)


In [ ]:
plt.style.use("dark_background")
_low, _high = 16000, 22000
ave_bin = 40
t_ave = np.arange(0, updated_spike_mat.shape[-1], ave_bin)
plt.figure(figsize=(14, 7))
_start = behav_df.TrialStartAligned[0] * ms
plt.subplot(221)
plt.title('cluster 3 (51-75)')
plt.plot(t_ave, updated_spike_mat[int(2*stim_clust_size):int(3*stim_clust_size)].mean(axis=0).reshape(-1, ave_bin).mean(axis=-1), c='C0', label=f'reward={behav_df.Rewarded[0]}')
plt.axvline(_start + behav_df.StimulusOnset[0] * ms, c='C1')
plt.xlim(_start, _high)
plt.legend(frameon=0)

plt.subplot(222)
plt.title('cluster 2 (26-50)')
plt.plot(t_ave, updated_spike_mat[stim_clust_size:int(2*stim_clust_size)].mean(axis=0).reshape(-1, ave_bin).mean(axis=-1), c='C0', label=f'reward={behav_df.Rewarded[0]}')
plt.axvline(_start + behav_df.ResponseEnd[0] * ms, c='C2')
plt.xlim(_start, _high)
plt.legend(frameon=0)


_start = behav_df.TrialStartAligned[1] * ms
plt.subplot(223)
plt.title('cluster 4 (76-100)')
plt.plot(t_ave, updated_spike_mat[int(3*stim_clust_size):].mean(axis=0).reshape(-1, ave_bin).mean(axis=-1), c='C3', label=f'reward={behav_df.Rewarded[1]}')
plt.axvline(_start + behav_df.StimulusOnset[1] * ms, c='C1')
plt.xlim(_start, 32000)
plt.legend(frameon=0)

plt.subplot(224)
plt.title('cluster 1 (1-25)')
plt.plot(t_ave, updated_spike_mat[:stim_clust_size].mean(axis=0).reshape(-1, ave_bin).mean(axis=-1), c='C3', label=f'reward={behav_df.Rewarded[1]}')
plt.axvline(_start + behav_df.ResponseEnd[1] * ms, c='C2')
plt.xlim(_start, 32000)
plt.legend(frameon=0)

plt.savefig('cluster_aves.png', dpi=200)

In [ ]:
cellbase_dir = '/home/mud/Workspace/ott_neuropix_data/toybase/'
# dump(updated_spike_mat, cellbase_dir + 'toy_spikes.npy', compress=3)
# dump(behav_df, cellbase_dir + "toy_behav_df", compress=3)